In [31]:
import os
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric, Dataset,concatenate_datasets, set_caching_enabled, ClassLabel
import pandas as pd

import random
from IPython.display import display, HTML

import json
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2ForCTC,Wav2Vec2Processor,Trainer,TrainingArguments,Wav2Vec2FeatureExtractor

import re
set_caching_enabled(False)

import soundfile as sf
import torchaudio


import IPython.display as ipd

import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from tqdm import tqdm
import torch

# Set environment variables
os.environ['WANDB_DISABLED '] = 'True'

import warnings
warnings.filterwarnings('ignore')

In [32]:
import logging
import transformers
transformers.logging.get_verbosity = lambda: logging.NOTSET
transformers.logging.get_verbosity()

0

In [36]:
import datasets
datasets.logging.get_verbosity = lambda: logging.NOTSET

In [37]:
import glob 
data_dir  = "/workspace/data_processed/*"
data_dir_list = glob.glob(data_dir)
data_dir_list.remove('/workspace/data_processed/5')

In [38]:
data_dir_list

['/workspace/data_processed/26',
 '/workspace/data_processed/13',
 '/workspace/data_processed/2',
 '/workspace/data_processed/7',
 '/workspace/data_processed/14',
 '/workspace/data_processed/24',
 '/workspace/data_processed/3',
 '/workspace/data_processed/38',
 '/workspace/data_processed/28',
 '/workspace/data_processed/16',
 '/workspace/data_processed/30',
 '/workspace/data_processed/36',
 '/workspace/data_processed/4',
 '/workspace/data_processed/18',
 '/workspace/data_processed/29',
 '/workspace/data_processed/8',
 '/workspace/data_processed/31',
 '/workspace/data_processed/27',
 '/workspace/data_processed/6',
 '/workspace/data_processed/10',
 '/workspace/data_processed/21',
 '/workspace/data_processed/15',
 '/workspace/data_processed/35',
 '/workspace/data_processed/9',
 '/workspace/data_processed/25',
 '/workspace/data_processed/22',
 '/workspace/data_processed/33',
 '/workspace/data_processed/20',
 '/workspace/data_processed/12',
 '/workspace/data_processed/39',
 '/workspace/data

In [39]:
data_set_list_total = []
for i in range(len(data_dir_list)):
    common_voice_val = Dataset.load_from_disk(data_dir_list[i])
    data_set_list_total.append(common_voice_val)

In [40]:
len(data_set_list_total)

38

In [41]:
len((data_set_list_total[:-1]))

37

In [42]:
len((data_set_list_total[-1:]))

1

In [43]:
common_voice_train = concatenate_datasets(data_set_list_total[:-1])

In [44]:
common_voice_test = concatenate_datasets(data_set_list_total[-1:])

In [45]:
common_voice_train

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 323525
})

In [46]:
common_voice_test

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 10000
})

In [47]:
from huggingface_hub import notebook_login
notebook_login()

In [48]:
repo_name = "wav2vec2-large-xlsr-300m-nepali"


In [49]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained("prajin/wav2vec2-large-xlsr-300m-nepali")

model = Wav2Vec2ForCTC.from_pretrained("prajin/wav2vec2-large-xlsr-300m-nepali")

loading feature extractor configuration file https://huggingface.co/prajin/wav2vec2-large-xlsr-300m-nepali/resolve/main/preprocessor_config.json from cache at /workspace/.cache/huggingface/transformers/465d49df6f91d98bc84913eabeca96e7faded8126da33aea29f247d071df8c45.cb58815f408ab0080376e53c13caeba88f343f0b26e9bc8dab3f2c7dd4bb074a
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2ProcessorWithLM",
  "return_attention_mask": true,
  "sampling_rate": 16000
}

loading file https://huggingface.co/prajin/wav2vec2-large-xlsr-300m-nepali/resolve/main/vocab.json from cache at /workspace/.cache/huggingface/transformers/3c208cfcb3d6ca41179c3373fba3ba5dba63039711ba931da1091572c592214a.a8c33b976c750da4e1fa62155883266dcd1c72126250503a0bec9c45b88a82c8
loading file https://huggingface.co/prajin/wav2vec2-large-xlsr-300m-nepali/r

In [50]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [51]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [52]:
wer_metric = load_metric("wer")

In [53]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [54]:
len(processor.tokenizer)

62

In [55]:
model = Wav2Vec2ForCTC.from_pretrained("prajin/wav2vec2-large-xlsr-300m-nepali")

loading configuration file https://huggingface.co/prajin/wav2vec2-large-xlsr-300m-nepali/resolve/main/config.json from cache at /workspace/.cache/huggingface/transformers/657533dcd20d79c1fc9f347ce7557a30cb7f43cd962285da950ae11185c4de33.cfce7ef4835fca714dff6f89bef4a3dd4a4fe3ffcf2f15ea6c18fce7e6fa6a44
Model config Wav2Vec2Config {
  "_name_or_path": "prajin/wav2vec2-large-xlsr-300m-nepali",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",


In [56]:
model.freeze_feature_extractor()

In [57]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=32,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=20,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=5e-5,
  warmup_steps=1000,
  save_total_limit=2,
  push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [58]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

/workspace/wav2vec2-large-xlsr-300m-nepali is already a clone of https://huggingface.co/prajin/wav2vec2-large-xlsr-300m-nepali. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp half precision backend


In [59]:
# model

In [60]:
trainer.train()

***** Running training *****
  Num examples = 323525
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 2
  Total optimization steps = 101100


RuntimeError: CUDA out of memory. Tried to allocate 5.70 GiB (GPU 0; 31.75 GiB total capacity; 28.57 GiB already allocated; 1.59 GiB free; 28.81 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [2]:
import torch
torch.cuda.empty_cache()


In [ ]:
!gpustat --show-pid

In [70]:
!kill 0